## Content

1. [Methods](#Methods)
2. [Problem to QUBO](#Problem-to-QUBO)
3. [Exact solver](#Exact-solver)
4. [Case n=3](#Case-n=3)
5. [Case n=6](#Case-n=6)
6. [Case n=12](#Case-n=12)
7. [Case n=144](#Case-n=144)

### Methods

In [1]:
import numpy as np
from pyqubo import Binary as bit
import dimod
from dwave.system import DWaveSampler
from dwave.system import EmbeddingComposite
import dwave.inspector
from dwave.system import LeapHybridSampler

### Problem to QUBO

In [2]:
p1, p2, p3 = 30.0, 10.0, 90.0
N = 100.0

In [3]:
q1, q2, q3 = bit("q1"), bit("q2"), bit("q3")

In [4]:
function = (- N + p1 * q1 + p2 * q2 +  p3 * q3)**2
model = function.compile()

In [5]:
L = model.to_qubo()

### Exact solver

In [6]:
ex_sampler = dimod.ExactSolver()
qubo_sample_set = ex_sampler.sample_qubo(L[0])

In [7]:
print(qubo_sample_set)

  q1 q2 q3   energy num_oc.
4  0  1  1 -10000.0       1
3  0  0  1  -9900.0       1
2  1  0  1  -9600.0       1
5  1  1  1  -9100.0       1
6  1  1  0  -6400.0       1
1  1  0  0  -5100.0       1
7  0  1  0  -1900.0       1
0  0  0  0      0.0       1
['BINARY', 8 rows, 8 samples, 3 variables]


In [8]:
bqm_3_qb = model.to_bqm()
bqm_sample_set = ex_sampler.sample(bqm_3_qb)
print(bqm_sample_set)

  q1 q2 q3  energy num_oc.
6  0  1  1     0.0       1
1  0  0  1   100.0       1
2  1  0  1   400.0       1
5  1  1  1   900.0       1
4  1  1  0  3600.0       1
3  1  0  0  4900.0       1
7  0  1  0  8100.0       1
0  0  0  0 10000.0       1
['BINARY', 8 rows, 8 samples, 3 variables]


#### Hybrid solver

In [9]:
hb_sampler = LeapHybridSampler(solver={'category': 'hybrid'})

In [10]:
qpu_sampler = DWaveSampler(solver={'topology__type': 'pegasus'})

In [11]:
qpu_sampler.properties['chip_id']

'Advantage_system6.4'

In [12]:
pegasus_sampler = EmbeddingComposite(qpu_sampler)

In [24]:
cs =  100000

### Case n=3

In [25]:
nr = 10
at = 4
sample_results = pegasus_sampler.sample(bqm_3_qb, num_reads=nr, annealing_time=at, chain_strength =cs,label='qubo_3')

In [14]:
print(sample_results)

  q1 q2 q3 energy num_oc. chain_.
0  0  1  1    0.0       5     0.0
1  0  0  1  100.0       4     0.0
2  1  0  1  400.0       1     0.0
['BINARY', 3 rows, 10 samples, 3 variables]


In [26]:
dwave.inspector.show(sample_results)

Serving Inspector on http://127.0.0.1:18000/?problemId=d75acec6-aad7-4d86-938e-5a3554ac6843

'http://127.0.0.1:18000/?problemId=d75acec6-aad7-4d86-938e-5a3554ac6843'

In [16]:
tl = 3
hb_name = f'hb_qubo_{3}_t{tl}'
hb_3_qb_results = hb_sampler.sample(bqm_3_qb, time_limit=tl, label=hb_name)

In [17]:
print(hb_3_qb_results)

  q1 q2 q3 energy num_oc.
0  0  1  1    0.0       1
['BINARY', 1 rows, 1 samples, 3 variables]


### Case n=6

In [18]:
set_seed = 3

In [19]:
add_qubits = 3

np.random.seed(set_seed)

qubits = [bit("q1"), bit("q2"), bit("q3")]

asset_6_prices = [p1, p2, p3]
function = -N + p1 * qubits[0] + p2 * qubits[1] +  p3 * qubits[2]

for i in range(add_qubits):
    
    delta = np.random.uniform(size=1)
    price = np.ceil(200 * delta[0]) 
    asset_6_prices.append(price)
    
    qubits.append(bit(f"q{4 + i}"))
    
    function += price * bit(f"q{4 + i}")
    
square_function = function**2

In [20]:
asset_6_prices

[30.0, 10.0, 90.0, 111.0, 142.0, 59.0]

In [21]:
model_6_qb = square_function.compile()
bqm_6_qb = model_6_qb.to_bqm()

In [51]:
nr = 10
at = 5
name = f"qubo_{nr}_t{at}"
sample_results_6_qb = pegasus_sampler.sample(bqm_6_qb, num_reads=nr, annealing_time=at, chain_strength =cs,label=name)

In [52]:
print(sample_results_6_qb)

  q1 q2 q3 q4 q5 q6  energy num_oc. chain_.
0  0  1  1  0  0  0     0.0       1     0.0
1  1  0  1  0  0  0   400.0       1     0.0
2  0  1  1  0  0  1  3481.0       1     0.0
3  1  0  0  0  1  0  5184.0       1     0.0
4  1  1  0  0  1  0  6724.0       1     0.0
5  0  0  0  0  1  1 10201.0       1     0.0
6  0  1  1  1  0  0 12321.0       1     0.0
7  0  0  1  0  1  0 17424.0       1     0.0
8  0  1  1  0  1  0 20164.0       1     0.0
9  1  0  1  0  1  1 48841.0       1     0.0
['BINARY', 10 rows, 10 samples, 6 variables]


In [53]:
dwave.inspector.show(sample_results_6_qb)

Serving Inspector on http://127.0.0.1:18000/?problemId=020ec3e3-7ec4-4d42-854e-65c99e98f3ca

'http://127.0.0.1:18000/?problemId=020ec3e3-7ec4-4d42-854e-65c99e98f3ca'

In [54]:
tl = 3
hb_name = f'hb_qubo_{len(asset_6_prices)}_t{tl}'
hb_6_qb_results = hb_sampler.sample(bqm_6_qb, time_limit=tl, label=hb_name)

In [55]:
print(hb_6_qb_results)

  q1 q2 q3 q4 q5 q6 energy num_oc.
0  0  1  1  0  0  0    0.0       1
['BINARY', 1 rows, 1 samples, 6 variables]


### Case n=12

In [56]:
add_qubits = 9

np.random.seed(set_seed)

qubits = [bit("q1"), bit("q2"), bit("q3")]

asset_12_prices = [p1, p2, p3]
function = -N + p1 * qubits[0] + p2 * qubits[1] +  p3 * qubits[2]

for i in range(add_qubits):
    
    delta = np.random.uniform(size=1)
    price = np.ceil(200 * delta[0]) 
    asset_12_prices.append(price)
    
    qubits.append(bit(f"q{4 + i}"))
    
    function += price * bit(f"q{4 + i}")
    
square_function = function**2

In [57]:
asset_12_prices

[30.0, 10.0, 90.0, 111.0, 142.0, 59.0, 103.0, 179.0, 180.0, 26.0, 42.0, 11.0]

In [58]:
model_12_qb = square_function.compile()
bqm_12_qb = model_12_qb.to_bqm()

In [71]:
nr = 10
at = 6
name = f"qubo_{nr}_t{at}"
sample_results_12_qb = pegasus_sampler.sample(bqm_12_qb, num_reads=nr, annealing_time=at, chain_strength =cs,label=name)

In [72]:
print(sample_results_12_qb)

  q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc. chain_.
0  1   0   0   1  0  0  0  0  1  0  0  0    0.0       2     0.0
1  1   0   0   1  1  0  0  0  1  0  0  0  100.0       1     0.0
2  1   0   1   1  0  0  0  0  0  0  0  0  289.0       1     0.0
3  1   0   1   0  0  0  0  0  1  0  0  0  961.0       1     0.0
4  1   0   1   1  0  0  0  0  1  0  0  0 1764.0       1     0.0
5  0   0   1   1  0  1  0  0  0  0  0  0 1849.0       1     0.0
6  1   1   1   0  0  0  0  0  1  0  0  0 3249.0       1     0.0
7  0   0   0   0  0  0  0  0  1  1  0  0 3844.0       1     0.0
8  1   1   1   1  0  1  0  0  0  0  0  0 9801.0       1     0.0
['BINARY', 9 rows, 10 samples, 12 variables]


In [73]:
dwave.inspector.show(sample_results_12_qb)

Serving Inspector on http://127.0.0.1:18000/?problemId=b4fdb3fe-a803-4952-9053-c9238f8a99cc

'http://127.0.0.1:18000/?problemId=b4fdb3fe-a803-4952-9053-c9238f8a99cc'

In [76]:
tl = 3
hb_name = f'hb_qubo_{len(asset_12_prices)}_t{tl}'
hb_12_qb_results = hb_sampler.sample(bqm_12_qb, time_limit=tl, label=hb_name)

In [77]:
print(hb_12_qb_results)

  q1 q10 q11 q12 q2 q3 q4 q5 q6 q7 q8 q9 energy num_oc.
0  1   0   0   1  0  0  0  0  1  0  0  0    0.0       1
['BINARY', 1 rows, 1 samples, 12 variables]


In [78]:
asset_12_prices

[30.0, 10.0, 90.0, 111.0, 142.0, 59.0, 103.0, 179.0, 180.0, 26.0, 42.0, 11.0]

### Case n=144

In [79]:
add_qubits = 141

np.random.seed(set_seed)

qubits = [bit("q1"), bit("q2"), bit("q3")]

asset_144_prices = [p1, p2, p3]
function = -N + p1 * qubits[0] + p2 * qubits[1] +  p3 * qubits[2]

for i in range(add_qubits):
    delta = np.random.uniform(size=1)
    price = np.ceil(200 * delta[0]) 
    asset_144_prices.append(price)
    
    qubits.append(bit(f"q{4 + i}"))
    
    function += price * bit(f"q{4 + i}")
    
square_function = function**2

In [80]:
print(asset_144_prices)

[30.0, 10.0, 90.0, 111.0, 142.0, 59.0, 103.0, 179.0, 180.0, 26.0, 42.0, 11.0, 89.0, 6.0, 92.0, 130.0, 56.0, 136.0, 119.0, 5.0, 112.0, 52.0, 84.0, 57.0, 139.0, 89.0, 32.0, 109.0, 157.0, 62.0, 45.0, 78.0, 188.0, 196.0, 135.0, 181.0, 170.0, 76.0, 19.0, 131.0, 112.0, 73.0, 46.0, 82.0, 94.0, 54.0, 59.0, 92.0, 173.0, 118.0, 57.0, 56.0, 91.0, 42.0, 41.0, 103.0, 18.0, 97.0, 73.0, 142.0, 150.0, 139.0, 138.0, 75.0, 134.0, 68.0, 115.0, 66.0, 90.0, 13.0, 49.0, 195.0, 47.0, 139.0, 131.0, 145.0, 96.0, 120.0, 14.0, 15.0, 40.0, 31.0, 21.0, 26.0, 111.0, 38.0, 191.0, 137.0, 109.0, 142.0, 53.0, 186.0, 168.0, 146.0, 97.0, 169.0, 149.0, 133.0, 183.0, 127.0, 74.0, 111.0, 40.0, 39.0, 146.0, 157.0, 195.0, 171.0, 109.0, 18.0, 98.0, 186.0, 158.0, 98.0, 92.0, 44.0, 36.0, 15.0, 179.0, 129.0, 29.0, 83.0, 10.0, 42.0, 147.0, 131.0, 96.0, 55.0, 131.0, 192.0, 88.0, 15.0, 12.0, 17.0, 192.0, 109.0, 168.0, 35.0, 53.0, 139.0, 180.0, 69.0, 13.0, 173.0]


In [81]:
model_144_qb = square_function.compile()
bqm_144_qb = model_144_qb.to_bqm()

In [82]:
nr = 10
at = 20
name = f"qubo_{nr}_t{at}"
sample_results_144_qb = pegasus_sampler.sample(bqm_144_qb, num_reads=nr, annealing_time=at, chain_strength=cs,label=name)

In [83]:
print(sample_results_144_qb)

  q1 q10 q100 q101 q102 q103 q104 q105 q106 q107 ... q99    energy num_oc. ...
0  0   1    0    0    0    0    1    0    0    0 ...   0 1483524.0       1 ...
7  0   1    0    0    0    0    0    0    0    0 ...   0 1758276.0       1 ...
1  0   1    0    0    0    0    1    0    0    0 ...   0 1811716.0       1 ...
4  0   1    0    0    0    0    0    0    0    0 ...   0 1811716.0       1 ...
3  0   0    0    0    0    0    0    0    0    0 ...   0 1819801.0       1 ...
5  0   0    0    0    1    0    1    0    0    0 ...   0 1915456.0       1 ...
9  0   0    0    0    0    0    1    0    0    0 ...   0 2076481.0       1 ...
2  0   1    0    0    1    0    1    0    0    0 ...   0 2660161.0       1 ...
8  0   1    0    0    0    0    1    0    0    0 ...   0 2666689.0       1 ...
6  0   1    0    0    0    0    0    0    0    0 ...   0 2805625.0       1 ...
['BINARY', 10 rows, 10 samples, 144 variables]


In [96]:
sample_results_144_qb.data_vectors

{'energy': array([1483524., 1811716., 2660161., 1819801., 1811716., 1915456.,
        2805625., 1758276., 2666689., 2076481.]),
 'num_occurrences': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'chain_break_fraction': array([0.44444444, 0.44444444, 0.44444444, 0.44444444, 0.45138889,
        0.4375    , 0.47916667, 0.43055556, 0.45138889, 0.46527778])}

In [85]:
dwave.inspector.show(sample_results_144_qb)

Serving Inspector on http://127.0.0.1:18000/?problemId=2488e944-a594-4796-98c5-ff82ebf2c803

'http://127.0.0.1:18000/?problemId=2488e944-a594-4796-98c5-ff82ebf2c803'

In [86]:
qubits = [i for i in sample_results_144_qb.first]
solution = [ (k, qubits[0][k]) for k in qubits[0] if qubits[0][k] !=0 ]

In [87]:
solution_prices = [asset_144_prices[int(s[0].split('q')[1])] for s in solution]
print(solution_prices)

[42.0, 146.0, 186.0, 15.0, 131.0, 15.0, 35.0, 92.0, 180.0, 112.0, 19.0, 142.0, 112.0, 54.0, 49.0, 139.0, 131.0, 26.0, 111.0]


In [88]:
tl = 3
hb_name = f'hb_qubo_{len(asset_144_prices)}_t{tl}'
hb_sample_results = hb_sampler.sample(bqm_144_qb, time_limit=tl, label=hb_name)

In [46]:
print(hb_sample_results)

  q1 q10 q100 q101 q102 q103 q104 q105 q106 q107 q108 ... q99 energy num_oc.
0  0   1    0    0    0    0    1    0    0    0    0 ...   0    0.0       1
['BINARY', 1 rows, 1 samples, 144 variables]


In [89]:
qubits = [i for i in hb_sample_results.first]
solution = [ (k, qubits[0][k]) for k in qubits[0] if qubits[0][k] !=0 ]
solution

[('q1', 1), ('q121', 1), ('q123', 1), ('q133', 1), ('q14', 1), ('q143', 1)]

In [92]:
solution_prices = [(s[0],asset_144_prices[int(s[0].split('q')[1])-1]) for s in solution]
print(solution_prices)

[('q1', 30.0), ('q121', 29.0), ('q123', 10.0), ('q133', 12.0), ('q14', 6.0), ('q143', 13.0)]
